In [1]:
!pip install keras.tuner

     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for keras.tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78935 sha256=a9e749b007cd9e88a93c2bc5f02f0164d2a4654b5b6f7341e255af8d8cd016bf
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=18e6c634ee0fab3556858b7759344380fef84b2fdeb0c2f0ad6b30a25a7fadeb
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras.tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion = keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels),(test_images, test_labels) = fashion.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
# Scaling
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.906166672706604

Best val_accuracy So Far: 0.9070000052452087
Total elapsed time: 00h 03m 52s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 112)       2912      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        64576     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1486896   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,554,874
Trainable params: 1,554,874
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1722 - accuracy: 0.9361 - val_loss: 0.2552 - val_accuracy: 0.9085
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1387 - accuracy: 0.9480 - val_loss: 0.2710 - val_accuracy: 0.9092
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1101 - accuracy: 0.9586 - val_loss: 0.2862 - val_accuracy: 0.9137
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0875 - accuracy: 0.9672 - val_loss: 0.3597 - val_accuracy: 0.9068
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0674 - accuracy: 0.9751 - val_loss: 0.3587 - val_accuracy: 0.9118
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0552 - accuracy: 0.9800 - val_loss: 0.3866 - val_accuracy: 0.9088
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0437 - accuracy: 0.9837 - val_loss: 0.4614 - val_accuracy